# Earlier Pre-processing

In [68]:
DATA_PATH = Path('../../data/interim')
df = pd.read_csv(DATA_PATH/'train_valid_data.csv', low_memory=False)
df.head()

,store,state,date,max_temperature_c,mean_temperature_c,min_temperature_c,dew_point_c,mean_dew_point_c,min_dew_point_c,max_humidity,...,promo_interval,day_of_week,sales,customers,open,promo,state_holiday,school_holiday,trend,week_start
0,1,HE,2013-01-01,8,6,3,6,3,1,93,...,None,1,0.0,0.0,0.0,0.0,a,1.0,61,2012-12-30
1,56,HE,2013-01-01,8,6,3,6,3,1,93,...,"Mar,Jun,Sept,Dec",1,0.0,0.0,0.0,0.0,a,1.0,61,2012-12-30
2,69,HE,2013-01-01,8,6,3,6,3,1,93,...,"Jan,Apr,Jul,Oct",1,0.0,0.0,0.0,0.0,a,1.0,61,2012-12-30
3,77,HE,2013-01-01,8,6,3,6,3,1,93,...,"Jan,Apr,Jul,Oct",1,0.0,0.0,0.0,0.0,a,1.0,61,2012-12-30
4,111,HE,2013-01-01,8,6,3,6,3,1,93,...,"Jan,Apr,Jul,Oct",1,0.0,0.0,0.0,0.0,a,1.0,61,2012-12-30


In [ ]:
df.dtypes[df.dtypes == 'object']
# state, events, store_type, assortment, promo_interval, state_holiday
# date we can skip because we'll use add_datepart() on that, and it will get turned into datetime (and many other things)
# week_start we can delete as that will get taken into account by add_datepart() for date

In [ ]:
#Date calculations to ensure we get the correct dataframes for testing and for training/validation
#df.date.min() # 2013-01-01
#df.date.max() # 2015-07-31
#pd.to_datetime('2015-07-31').dayofweek # 4 = Friday
#pd.to_datetime('2015-07-31').week # 31
#pd.to_datetime('2015-06-19').dayofweek # 4
#pd.to_datetime('2015-06-19').week # 25
#pd.to_timedelta(pd.to_datetime('2015-07-31') - pd.to_datetime('2015-06-19'))
#test_data = df.loc[df.date >= '2015-06-20'].copy()
#len(test_data) # 46830
#len(test_data)/1115 # 42 days
#train_valid_data = df.loc[df.date <= '2015-06-19'].copy()
#len(train_valid_data) # 1003500
#len(train_valid_data)/1115 # 900 days

In [ ]:
# Figure out how big the validation set should be
#len(train_valid_data) # 1,003,500
#len(train_valid_data) * .2 # 200,700
#len(train_valid_data) * .2 / 1115 # 180 days = 20% of the train_valid_data set

In [ ]:
test_data.to_csv('../../data/interim/test_data.csv', index=False)
print('did test')
train_valid_data.to_csv('../../data/interim/train_valid_data.csv', index=False)
print('did train')

# Imports

In [ ]:
from fastai.tabular import *
import pandas as pd
from pathlib import Path

# Get Data

In [ ]:
DATA_PATH = Path('../../data/interim')

In [92]:
# Import train_valid_data and test_data dataframes
train_valid_df = pd.read_csv(DATA_PATH/'train_valid_data.csv', low_memory=False)
test_df = pd.read_csv(DATA_PATH/'test_data.csv', low_memory=False)

# Set up preprocessing
# Category names, to which we'll apply embeddings
cat_names = ['assortment', 'events', 'promo_interval', 'state', 'state_holiday', 'store_type']
# Won't do anything with cont_names now but for completeness
# Also note we intentionally left out week_start as that will get taken into account by add_datepart() for date
cont_names = ['cloud_cover', 'competition_distance', 'competition_open_since_month', 'competition_open_since_year',
              'customers', 'day_of_week', 'dew_point_c', 'max_gust_speed_km_h', 'max_humidity',
              'max_sea_level_pressureh_pa', 'max_temperature_c', 'max_visibility_km', 'max_wind_speed_km_h',
              'mean_dew_point_c', 'mean_humidity', 'mean_sea_level_pressureh_pa', 'mean_temperature_c',
              'mean_visibility_km', 'mean_wind_speed_km_h', 'min_dew_point_c', 'min_humidity', 'min_sea_level_pressureh_pa',
              'min_temperature_c', 'min_visibility_km', 'open', 'precipitationmm', 'promo', 'promo2', 'promo2_since_week',
              'promo2_since_year', 'school_holiday', 'store', 'trend', 'wind_dir_degrees']

# We can define a bunch of Transforms that will be applied to our variables. 
# Here we transform all categorical variables into categories. 
# We also replace missing values for continuous variables by the median column value and normalize those.
# And we add columns relevant to the date: ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear', 'Is_month_end',
#                             'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
# Note that FillMissing should do nothing since we don't have any NaNs -- leaving it in anyway
procs = [FillMissing, Categorify, Normalize]#,
         #add_datepart(train_valid_df, 'date', drop=True, time=False)]#,
         #add_datepart(test_df, 'date', drop=True, time=False)]

valid_idx = range(len(df)-200700, len(df))
data = TabularDataBunch.from_df(path=DATA_PATH, df=df, test_df=test_df, dep_var='sales', valid_idx=valid_idx,
                                     procs=procs, cat_names=cat_names, cont_names=cont_names, bs=64, num_workers=4)
print('done')

/home/ubuntu/anaconda3/lib/python3.6/site-packages/fastai/tabular/transform.py:63: RuntimeWarning: Values and categories have different dtypes. Did you mean to use
'Categorical.from_codes(codes, categories)'?
  df.loc[:,n] = pd.Categorical(df[n], categories=self.categories[n], ordered=True)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/fastai/tabular/transform.py:63: RuntimeWarning: None of the categories were found in values. Did you mean to use
'Categorical.from_codes(codes, categories)'?
  df.loc[:,n] = pd.Categorical(df[n], categories=self.categories[n], ordered=True)


done


In [82]:
data = testorama
(cat_x,cont_x),y = next(iter(data.train_dl))
for o in (cat_x, cont_x, y): print(to_np(o[:5]))

[[  1 510   1   4   7   1   1]
 [  1 344  11   2   2   1   4]
 [  3 651   2   1   2   1   4]
 [  1  88  15   4   1   3   1]
 [  1 745  12   2   8   1   1]]
[[-1.544212 -0.437269  1.446941 -0.924744 ... -0.447853  1.656357 -0.97199   0.986241]
 [ 0.380457 -0.508535  0.006039 -0.026026 ... -0.447853 -0.860089  0.639254 -1.002204]
 [ 1.022013  0.613243  0.006039 -0.026026 ... -0.447853  0.231196 -0.213757  0.922441]
 [ 1.022013 -0.589039 -1.224722  0.807686 ...  2.232873 -1.379453  0.165359  1.847547]
 [ 0.380457 -0.672182  1.828606 -0.732252 ... -0.447853 -1.685508  0.354917  0.13557 ]]
[   0. 4228.    0.    0. 7481.]


In [93]:
emb_szs = {'assortment': 2, 'events': 11, 'promo_interval': 2, 'state': 6, 'state_holiday': 2, 'store_type': 2} 

learn = tabular_learner(data, layers=[200,100], emb_szs=emb_szs, metrics=accuracy)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy


RuntimeError: Expected object of scalar type Long but got scalar type Float for argument #2 'other'

In [ ]:
learn.predict(df.iloc[0])